In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt

import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu

import Models.Soybean_USA_Yield_GA as sy

import APIs.Bloomberg as ba

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [39]:
scope = sy.Define_Scope()
raw_data = sy.Get_Data_All_Parallel(scope)    

In [40]:
m=um.pick_model('GA_soy',72)
ww = um.from_cols_to_var_windows(m.params.index)

Start Deserializing: GA_soy
Done Deserializing: GA_soy


In [41]:
model_df = um.extract_yearly_ww_variables(w_df = raw_data['w_w_df_all']['hist'],var_windows= ww)
model_df['year']=model_df.index

In [42]:
y_df = raw_data['yield'][raw_data['yield'].index<2022][['Yield']]

In [43]:
X_df=model_df

folds = um.folds_expanding(X_df, min_train_size=10)
# um.print_folds(folds = folds, years=train_df.index.values)

# Cross-Validation calculation
cv_score = um.stats_model_cross_validate(X_df, y_df, folds)

In [5]:
m.params

const                     -960.550291
year                         0.509614
USA_TempMax_Aug21-Aug28     -0.383391
USA_Sdd30_Jun17-Aug19        1.804934
USA_TempMax_Aug16-Aug19     -0.348013
USA_Sdd30_Jul04-Aug20       -0.909144
USA_Sdd30_Jun27-Jul15        0.800158
dtype: float64

In [44]:
np.mean(cv_score['cv_p'])

0.1864557888779011

In [25]:
f='GA_soy'
dm_best = {}
dm_best[f] = uu.deserialize(f)
r=dm_best[f]

r.keys()

Start Deserializing: GA_soy
Done Deserializing: GA_soy


dict_keys(['best_fitness', 'model', 'MAE', 'MAPE', 'fitness', 'corr', 'cv_p', 'cv_r_sq', 'cv_MAE', 'cv_MAPE', 'cv_corr'])

In [37]:
np.mean(r['cv_p'][72])

0.001679548798393492

In [ ]:
print(len(r['model']))
print(len(r['corr']))
print(r['cv_p'][0])